In [ ]:
# !pip install langchain faiss-cpu sentence-transformers openai langchain_community

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "______"  # Your key from OpenRouter, Groq, etc.

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model="shisa-ai/shisa-v2-llama3.3-70b:free",
    temperature=0,
    max_tokens=512
)


/tmp/ipython-input-2-2268065491.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("ocr_text_with_tables.txt", "r", encoding="utf-8") as f:
    doc_text = f.read()

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
text_chunks = splitter.split_text(doc_text)

text_docs = [{"type": "text", "content": chunk, "source": "ocr_text"} for chunk in text_chunks]
print(f"[✓] {len(text_docs)} text chunks created.")


[✓] 94 text chunks created.


In [ ]:
import json
from hashlib import md5

def table_to_md(rows):
    md = []
    for i, row in enumerate(rows):
        line = "| " + " | ".join(str(cell) for cell in row) + " |"
        md.append(line)
        if i == 0:
            md.append("| " + " | ".join(["---"] * len(row)) + " |")
    return "\n".join(md)

table_docs = []

for fname in os.listdir("Altered_tables"):
    if fname.endswith(".json"):
        with open(f"Altered_tables/{fname}", "r", encoding="utf-8") as f:
            rows = json.load(f)
            table_md = table_to_md(rows)
            table_docs.append({
                "type": "table",
                "content": table_md,
                "source": fname
            })

print(f"[✓] {len(table_docs)} table chunks created.")


[✓] 10 table chunks created.


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

all_docs = []

for item in text_docs + table_docs:
    all_docs.append(Document(
        page_content=item["content"],
        metadata={"source": item["source"], "type": item["type"]}
    ))

db = FAISS.from_documents(all_docs, embedder)
print("[✓] All data embedded and indexed in FAISS.")


/tmp/ipython-input-5-245979736.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

[✓] All data embedded and indexed in FAISS.


In [ ]:
!pip install openai==0.28

import openai

# 🔑 Your OpenRouter API key goes here:
openai.api_key = "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"
openai.api_base = "https://openrouter.ai/api/v1"

# 🔁 Use the correct model name:
response = openai.ChatCompletion.create(
    model="shisa-ai/shisa-v2-llama3.3-70b:free",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of Japan?"}
    ]
)

print(response['choices'][0]['message']['content'])


The capital of Japan is **Tokyo**. 

It's also the largest city in Japan and one of the most populous metropolitan areas in the world! 😊


In [ ]:
def ask(query, k=5):
    rel_docs = db.similarity_search(query, k=k)

    context = "\n\n---\n\n".join([doc.page_content for doc in rel_docs])
    prompt = f"""You are an assistant answering questions using the provided documents only.
If the answer is not in the documents, say "I don't know based on the provided documents."

### Query:
{query}

### Documents:
{context}
"""
    res = llm.predict(prompt)
    return res


In [ ]:
response = ask("My grade is E6 and i reside in city classified as X what is my revised rate of house rent recovery ")
print(response)


/tmp/ipython-input-8-1518132603.py:14: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = llm.predict(prompt)


Based on the provided documents, your revised rate of house rent recovery is **Rs. 2750 per month**.  

Here’s the breakdown:  
- Your grade is **E6**, which falls under the **"E6 & above"** category.  
- Your city is classified as **X**, so the applicable rate is **Rs. 2750 per month** (as per the table in the documents).  

Note: The additional details about HRA percentages and minimum rates are not directly relevant to your specific query about house rent recovery for corporation-provided accommodation. The table in the documents provides the exact rate for your grade and city classification.


In [ ]:
response = ask("What does the document discuss about ?")
print(response)

The document discusses the following topics:  

1. **General Conditions** – Definitions of terms like "Corporation," "Competent Authority," "Pay," and "Family" for the purpose of pay revision rules.  
2. **Certification of Non-Occupation of Government/Corporation Accommodation** – A declaration form where an employee certifies that they or their spouse do not occupy any government or NHPC-provided accommodation.  
3. **Request for Retention of Accommodation/HRA Drawal** – A certification process for verifying eligibility for House Rent Allowance (HRA), ensuring employees have not been provided with corporate accommodation and have submitted required declarations.  
4. **Undertaking/Declaration for HRA** – Employees must submit an annual declaration (Annexure IV) confirming retention of family at the previous posting place while drawing HRA.  
5. **HRA on Superannuation** – The document briefly mentions HRA rules applicable after an employee’s retirement (superannuation).  

**Summary:*

In [ ]:
response = ask("I live in Delhi , accordingly what will be mu HRA rate per month  ?")
print(response)

Based on the provided documents, since Delhi is classified as an **X-Class city** (population of 50 Lakh and above), the HRA rate per month would be **24% of Basic Pay** as per the initial rates (Table 1).  

However, if the IDA (Industrial Dearness Allowance) crosses **25%**, the HRA rate would be revised to **27%** for X-Class cities (as per the revision rule). If the IDA crosses **50%**, the rate would further increase to **30%**.  

Since the current IDA status is not provided in the documents, the default HRA rate for Delhi would be **24% of Basic Pay**.  

**Final Answer:**  
**24% of Basic Pay** (or higher if IDA exceeds 25% or 50%, but no such information is given in the documents).  

*(Note: The minimum HRA rates of Rs. 5,400/- for X-Class cities apply if 24% of Basic Pay is lower than this amount.)*
